In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from pycaret.regression import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

In [2]:
input_csv_path = 'data/NY_HOURLY_RIDERSHIP_2022.csv'
output_csv_path = 'data/NY_LORDHELPME_RIDERSHIP_2022.csv'
holidays_csv_path = 'data/Holidays Dataset US.csv'

In [3]:
chunk_size = 10000
sample_start_date = '2023-01-01'
sample_end_date = '2023-12-31'

In [4]:
def map_hour_to_period(hour):
    if 1 <= hour <= 6:
        return 1
    elif 7 <= hour <= 12:
        return 2
    elif 13 <= hour <= 18:
        return 3
    elif 19 <= hour <= 24 or hour == 0:
        return 4

In [5]:
def preprocess_chunk(chunk_to_process, holiday_dates):
    chunk_to_process = chunk_to_process.copy()
    
    chunk_to_process['date'] = chunk_to_process['transit_timestamp'].dt.date
    chunk_to_process['hour'] = chunk_to_process['transit_timestamp'].dt.hour
    chunk_to_process['month'] = chunk_to_process['transit_timestamp'].dt.month
    chunk_to_process['weekday'] = chunk_to_process['transit_timestamp'].dt.weekday
    chunk_to_process['is_holiday'] = chunk_to_process['date'].apply(lambda x: x in holiday_dates)
    
    return chunk_to_process

In [6]:
holidays_dataset = pd.read_csv(holidays_csv_path)
holidays_dataset = holidays_dataset[holidays_dataset['Type'] == "['National holiday']"]
holidays_dataset['Date'] = pd.to_datetime(holidays_dataset['Date'], errors='coerce')
holiday_dates = set(holidays_dataset['Date'].dt.date)

In [7]:
chunk_iter = pd.read_csv(input_csv_path, chunksize=chunk_size, parse_dates=['transit_timestamp'], date_format='%m/%d/%Y %I:%M:%S %p')

sampled_chunks = []

for chunk in chunk_iter:
    preprocessed_chunk = preprocess_chunk(chunk, holiday_dates)

    if not preprocessed_chunk.empty:
        sampled_chunks.append(preprocessed_chunk)

In [8]:
df = pd.concat(sampled_chunks)

df.to_csv(output_csv_path, index=False)

In [9]:
df = pd.read_csv(output_csv_path)

KeyboardInterrupt: 

In [10]:
categorical_features = ['station_complex_id', 'weekday']
numeric_features = ['hour', 'is_holiday', 'month']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='poly'))
])

X = df.drop('ridership', axis=1)
y = df['ridership']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# {'poly', 'linear', 'sigmoid', 'rbf', 'precomputed'}

MemoryError: Unable to allocate 647. MiB for an array with shape (2, 42434192) and data type float64

In [ ]:
df = pd.read_csv(output_csv_path)

X = df[['hour', 'weekday', 'station_complex_id', 'is_holiday', 'month']]
y = df['ridership']

X = pd.get_dummies(X, columns=['weekday', 'station_complex_id'], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_features': ['auto', 'sqrt'],  # Number of features to consider at every split
    'max_depth': [10, 20, 30, None],  # Maximum number of levels in tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), 
                           param_grid=param_grid, 
                           cv=3,  # Number of folds in cross-validation
                           verbose=2,  # Controls the verbosity: the higher, the more messages
                           n_jobs=-1)  # Number of jobs to run in parallel


grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Improved Mean Squared Error: {mse}')
print(f'Improved R^2 Score: {r2}')

In [11]:
X = df.drop('ridership', axis=1)
y = df['ridership']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = X_train.copy()
train_data['ridership'] = y_train

setup(data = train_data, target = 'ridership', session_id=123)

best_model = compare_models()

MemoryError: Unable to allocate 405. MiB for an array with shape (1, 53042741) and data type int64

In [ ]:
tuned_model = tune_model(best_model)

In [ ]:
final_model = finalize_model(tuned_model)

In [ ]:
predictions = predict_model(final_model, data=X_test)

In [ ]:
plot_model(tuned_model, plot = 'residuals')

In [ ]:
print(predictions.head())

In [ ]:
mse = mean_squared_error(y_test, predictions['prediction_label'])
r2 = r2_score(y_test, predictions['prediction_label'])
rmse = np.sqrt(mean_squared_error(y_test, predictions['prediction_label']))
mae = mean_absolute_error(y_test, predictions['prediction_label'])
mape = np.mean(np.abs((y_test - predictions['prediction_label']) / y_test)) * 100

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Absolute Percentage Error: {mape}%')

In [ ]:
plot_model(final_model, plot='error', data=X_test, target=y_test)

In [ ]:
residuals = y_test - predictions['prediction_label']
plt.figure(figsize=(10,6))
plt.scatter(predictions['prediction_label'], residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

# Histogram of residuals
plt.figure(figsize=(10,6))
plt.hist(residuals, bins=30)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()


In [ ]:
scores = cross_val_score(final_model, X, y, scoring='neg_root_mean_squared_error', cv=5)
print(f'Cross-validated RMSEs: {-scores}')
print(f'Mean RMSE: {-np.mean(scores)}')